In [1]:
print("hello")

hello


In [ ]:
import json
import csv
from pathlib import Path

# Paths
ANNOTATION_FILE = 'annotations/person_keypoints_train2017.json'
IMAGE_DIR = 'train2017'
CSV_OUT = 'coco_keypoints_train.csv'

# Load JSON
with open(ANNOTATION_FILE, 'r') as f:
    coco = json.load(f)

# Build image_id → (filename, w, h) lookup
id_to_file = {img['id']: (img['file_name'], img['width'], img['height']) for img in coco['images']}

# Open CSV writer
with open(CSV_OUT, 'w', newline='') as f:
    writer = csv.writer(f)
    
    # Write header
    header = ['image_path'] + [f'x{i+1}' for i in range(17)] + [f'y{i+1}' for i in range(17)] + ['width', 'height']
    writer.writerow(header)
    
    """
    #for full dataset -----------------------
    # Loop over annotations
    for ann in coco['annotations']:
        if ann['num_keypoints'] < 5:
            continue  # Skip incomplete labels

        image_id = ann['image_id']
        keypoints = ann['keypoints']  # length 51

        filename, width, height = id_to_file[image_id]
        image_path = f"{IMAGE_DIR}/{filename}"

        xs = [keypoints[i] for i in range(0, len(keypoints), 3)]
        ys = [keypoints[i+1] for i in range(0, len(keypoints), 3)]

        row = [image_path] + xs + ys + [width, height]
        writer.writerow(row)
    """

    # for subset = 1000
    # max_rows = 56599
    max_rows = 10000
    written = 0

    for ann in coco['annotations']:
        if written >= max_rows:
            break  # stop writing after 1000 rows

        if ann['num_keypoints'] < 5:
            continue  # skip if insufficient joints

        image_id = ann['image_id']
        keypoints = ann['keypoints']
        filename, width, height = id_to_file[image_id]

        xs = [keypoints[i] for i in range(0, len(keypoints), 3)]
        ys = [keypoints[i+1] for i in range(0, len(keypoints), 3)]

        # row = [f"{IMAGE_DIR}/{filename}"] + xs + ys + [width, height]
        row = [f"{filename}"] + xs + ys + [width, height]
        writer.writerow(row)
        written += 1   
